In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from numpy import load
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.1 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/ILDC_expert/anno_dataset.csv')

In [4]:
import pickle
file_path = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/only_annotation/attention_anno/att_wt_of_LSGBERT_bigru_anno.pkl'
with open(file_path, 'rb') as file:
    att_scores_anno = pickle.load(file)

In [5]:
output_dir = '/content/drive/MyDrive/Thesis/calibration/saved_model_multi_2560'
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [6]:
def lsgbert_detok(tokens):
    # Initialize an empty list for the detokenized sequence
    detokenized_sequence = []

    # Initialize a variable for handling '##' signs
    current_token = ""

    # Iterate through the tokens in the sequence
    for token in tokens:
        if token.startswith("##"):
            # If the token starts with '##', remove '##' and append the rest of the token to the previous token
            current_token += token[2:]
        else:
            # If the token does not start with '##', check if it is a punctuation mark
            if current_token and token in string.punctuation:
                # If the current token is a punctuation mark, attach it to the previous token
                current_token += token
            else:
                # If the current token is not a punctuation mark, add the previous token (if present) to the detokenized sequence
                if current_token:
                    detokenized_sequence.append(current_token)
                # Add the current token to the detokenized sequence
                current_token = token

    # Make sure to add the last token to the detokenized sequence
    if current_token:
        detokenized_sequence.append(current_token)


    tokens_concat = []

    prev_tok=""

    for token in detokenized_sequence:
      if token.endswith('-'):
          prev_tok = token
      else:
          if (prev_tok != ""):
              token = prev_tok + token
              prev_tok = ""
          tokens_concat.append(token)

    if (prev_tok!=""):
        tokens_concat.append(prev_tok)

    # Join the tokens in the detokenized sequence into a string
    detokenized_text = " ".join(tokens_concat)

    return detokenized_text


In [7]:
max_indexes = []
for i in range(len(att_scores_anno)):
  index_max= att_scores_anno[i][0].argmax()
  max_indexes.append(index_max)

In [8]:
final_explanations_att = {}
for i in range(len(df)):
  all_toks = tokenizer.tokenize(df.iloc[i]['text'])
  if(len(all_toks) > 10000):
      all_toks = all_toks[len(all_toks)-10000:]
  splitted_toks = []
  l=0
  r=510
  while(l<len(all_toks)):
    splitted_toks.append(all_toks[l:min(r,len(all_toks))])
    l+=410
    r+=410

  explanation_of_this_doc = lsgbert_detok(splitted_toks[max_indexes[i]])
  doc_name = df.iloc[i]['name']
  final_explanations_att[doc_name] = explanation_of_this_doc

Token indices sequence length is longer than the specified maximum sequence length for this model (5458 > 4096). Running this sequence through the model will result in indexing errors


In [9]:
final_explanations_att

{'1960_12.txt': 'which was referred to the high court was whether there was material to hold that the sum of rs. 35, 807 was maintenance allowance, but the high court reformulated this question and after going into various documents it came to a conclusion different from that of the tribunal and it reversed the findings of the tribunal and answered the question in a manner suggestive of an appellate rather than advisory jurisdiction.',
 '1953_14.txt': 's given above that in this case the lis in the sense explained above arose before the date of amendment of the section. further, even if the lis is to be taken as arising only on the date of assessment, there was a possibility of such a lis arising as soon as proceedings started with the filing of the return or, at any rate, when the authority called for evidence and started the hearing and the right of appeal must be taken to have been in existence even at those dates. for the purposes of the accrual of the right of appeal the critical 

In [11]:
import json
json_file = open("/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/only_annotation/attention_anno/att_exp_anno.json", "w")
json.dump(final_explanations_att, json_file)